加载环境变量

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
# 配置 DeepSeek API 密钥
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
if not DEEPSEEK_API_KEY:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [2]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings # pip install langchain-huggingface

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


4. 创建向量存储

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['ee03c8eb-934e-405d-b556-38b25f7d4e83',
 'e15a84a6-525d-47d6-bc9f-7190ff447181',
 'deac0d7a-429c-4c1f-953d-e559411b7d97',
 '0ce8b6e7-4e6c-4306-b6fd-29a543d1b48e',
 '65511949-4159-424f-9fb4-c3a07cf7827c',
 'aee4283e-8f9b-43e6-bbf4-fad3f4db1035',
 '690dc2cf-675e-432e-99e5-1343d573f60f',
 'fa851a61-bcc1-4367-b6a9-ab2ba09cc45d',
 '1d0ef6f2-c1d9-4314-bbdc-811da9138323',
 '189b102b-f111-4320-b23a-0ec265e3513c',
 '613369ee-e6e6-4c18-b5aa-02520377647b',
 '4fa4d743-31fb-4c31-9903-41b5e84f3434',
 'f3514e20-a161-4c87-8358-de764bc78b41',
 'f129e3b2-306b-4d17-a281-c4920df5e3e1',
 '35bc82e0-d989-4b0a-a063-962671f67a29',
 '7806eda4-91f9-429d-be75-3cc0da645ac9',
 '9d570f3b-6d08-4b70-a18f-0a49af013eb0',
 'c6a88929-93d9-426b-b9d3-f34da22aef48',
 '057cd54b-acf3-4392-9a13-948f13e25dbd',
 '105fec33-e8e6-400c-b43e-98f776cecce2',
 'a92b2e2d-4bb9-4d6f-94ba-973feb406302',
 '307a6c95-4b7b-498a-9fbc-e4465e3482bd',
 'd04d0e59-e045-47dd-bf95-d77216c8fdb5',
 '355f91cf-5f10-437a-879b-f598f822202f',
 '6880c0aa-6ceb-

5. 构建用户查询

In [6]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [7]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用大语言模型生成答案

In [9]:
from langchain_deepseek import ChatDeepSeek # pip install langchain-deepseek

llm = ChatDeepSeek(
    model="deepseek-chat",  # DeepSeek API 支持的模型名称
    temperature=0.7,        # 控制输出的随机性
    max_tokens=2048,        # 最大输出长度
    api_key=DEEPSEEK_API_KEY  # 从环境变量加载API key
)

9. 生成答案

In [10]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
--------------------------------------------------
回答: 根据提供的上下文，《黑神话：悟空》的游戏场景融合了中国多地文化和自然地标，具体包括以下取景地：  
1. **重庆大足石刻**  
2. **山西省**的多个地点：  
   - 小西天  
   - 南禅寺  
   - 铁佛寺  
   - 广胜寺  
   - 鹳雀楼（游戏中的标志性地标之一）  
3. 其他未明确提及但通过文博日历和IGN报道提到的中国古建场景（如部分寺庙和自然景观）。  

此外，游戏地图设计包含线性进程与宽阔探索区域，场景会随剧情推进动态变化。更多细节可参考开发团队公开的取景地幕后花絮（如IGN中国报道）。
